# Surveil — Weapon Detector Training
Trains YOLO11n on real 1920×1080 CCTV footage (Pascal VOC XML annotations).

**Dataset:** `frankmurphy24/cctv-weapon-detector` (Images/ folder)  
**Classes:** `gun` (Handgun), `rifle` (Short_rifle), `knife` (Knife)  
**Output:** `/kaggle/working/runs/weapon/train/weights/best.pt`

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:  # cap at 5 per dir — Images/ has thousands of files
        print(os.path.join(dirname, filename))
    if filenames:
        print(f'  ... ({len(filenames)} files total in {dirname})')
        break  # stop after first dir with files to avoid flooding output

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images/Cam7-From09-05-50To10-09-24_Segment_8_x264_frame_805.jpg
/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images/Cam7-From09-05-50To10-09-24_Segment_8_x264_frame_3362.xml
/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images/Cam5-From09-27-50To09-29-30-Guns_x264_frame_140.jpg
/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images/Cam7-From09-05-50To10-09-24_Segment_8_x264_frame_1448.jpg
/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images/Cam1-From09-23-00To10-03-25_Segment_5_x264_frame_128.xml
  ... (10298 files total in /kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images)


In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00


In [3]:
import random
import shutil
import xml.etree.ElementTree as ET
from pathlib import Path
from tqdm.auto import tqdm

# ---------------------------------------------------------------------------
# Class mapping — normalise all XML name variants to 3 YOLO classes
# ---------------------------------------------------------------------------

CLASS_MAP: dict[str, tuple[int, str]] = {
    # gun (index 0)
    "handgun":    (0, "gun"),
    "Handgun":    (0, "gun"),
    "pistol":     (0, "gun"),
    "Pistol":     (0, "gun"),
    # rifle (index 1)
    "rifle":      (1, "rifle"),
    "Rifle":      (1, "rifle"),
    "short_rifle":(1, "rifle"),
    "Short_rifle":(1, "rifle"),
    # knife (index 2)
    "knife":      (2, "knife"),
    "Knife":      (2, "knife"),
    "machete":    (2, "knife"),
}

CLASSES = ["gun", "rifle", "knife"]

# ---------------------------------------------------------------------------
# Pascal VOC XML → YOLO txt
# ---------------------------------------------------------------------------

def voc_to_yolo(xml_path: Path) -> list[str]:
    """Parse a Pascal VOC XML and return normalised YOLO label lines.
    Returns [] for negative frames (no annotated objects).
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
    except ET.ParseError:
        return []

    size = root.find("size")
    if size is None:
        return []
    w = float(size.findtext("width") or 0)
    h = float(size.findtext("height") or 0)
    if w == 0 or h == 0:
        return []

    lines = []
    for obj in root.findall("object"):
        name = obj.findtext("name", "").strip()
        if name not in CLASS_MAP:
            continue
        cls_idx, _ = CLASS_MAP[name]

        bndbox = obj.find("bndbox")
        if bndbox is None:
            continue
        xmin = float(bndbox.findtext("xmin") or 0)
        ymin = float(bndbox.findtext("ymin") or 0)
        xmax = float(bndbox.findtext("xmax") or 0)
        ymax = float(bndbox.findtext("ymax") or 0)

        xmin, xmax = max(0, xmin), min(w, xmax)
        ymin, ymax = max(0, ymin), min(h, ymax)
        if xmax <= xmin or ymax <= ymin:
            continue

        cx = ((xmin + xmax) / 2) / w
        cy = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h
        lines.append(f"{cls_idx} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    return lines

# ---------------------------------------------------------------------------
# Dataset assembly
# ---------------------------------------------------------------------------

def collect_pairs(src_dir: Path) -> list[tuple[Path, Path]]:
    """Return (jpg_path, xml_path) pairs where both files exist."""
    pairs = []
    for xml_path in sorted(src_dir.glob("*.xml")):
        jpg_path = xml_path.with_suffix(".jpg")
        if jpg_path.exists():
            pairs.append((jpg_path, xml_path))
    print(f"  {src_dir.name}: {len(pairs)} image/annotation pairs found")
    return pairs


def convert_and_copy(
    pairs: list[tuple[Path, Path]],
    img_out_dir: Path,
    lbl_out_dir: Path,
) -> dict[str, int]:
    """Convert VOC XMLs to YOLO txts and copy images. Returns class counts."""
    img_out_dir.mkdir(parents=True, exist_ok=True)
    lbl_out_dir.mkdir(parents=True, exist_ok=True)

    counts = {"pos": 0, "neg": 0, **{c: 0 for c in CLASSES}}

    for jpg_path, xml_path in tqdm(pairs, desc="Converting annotations", unit="img"):
        lines = voc_to_yolo(xml_path)
        stem = f"{xml_path.parent.name}_{xml_path.stem}"
        shutil.copy(jpg_path, img_out_dir / f"{stem}.jpg")
        (lbl_out_dir / f"{stem}.txt").write_text("\n".join(lines))

        if lines:
            counts["pos"] += 1
            for line in lines:
                counts[CLASSES[int(line.split()[0])]] += 1
        else:
            counts["neg"] += 1

    return counts


def write_yaml(out_dir: Path, val_ratio: float = 0.1) -> Path:
    """Write YOLO dataset YAML with a random 90/10 train/val split."""
    img_dir = out_dir / "images" / "all"
    all_imgs = list(img_dir.glob("*.jpg"))
    random.shuffle(all_imgs)

    n_val = max(1, int(len(all_imgs) * val_ratio))
    val_imgs   = all_imgs[:n_val]
    train_imgs = all_imgs[n_val:]

    for split_name, split_imgs in [("train", train_imgs), ("val", val_imgs)]:
        split_img_dir = out_dir / "images" / split_name
        split_lbl_dir = out_dir / "labels" / split_name
        split_img_dir.mkdir(parents=True, exist_ok=True)
        split_lbl_dir.mkdir(parents=True, exist_ok=True)
        for img in tqdm(split_imgs, desc=f"Copying {split_name}", unit="img"):
            lbl = (out_dir / "labels" / "all" / img.name).with_suffix(".txt")
            dst_img = split_img_dir / img.name
            dst_lbl = split_lbl_dir / img.with_suffix(".txt").name
            if not dst_img.exists():
                shutil.copy(img, dst_img)
            if not dst_lbl.exists() and lbl.exists():
                shutil.copy(lbl, dst_lbl)

    yaml_path = out_dir / "weapon.yaml"
    yaml_path.write_text(f"""\
path: {out_dir.resolve()}
train: images/train
val:   images/val

nc: {len(CLASSES)}
names: {CLASSES}
""")
    print(f"  YAML: {yaml_path}")
    print(f"  Train: {len(train_imgs)} images")
    print(f"  Val:   {len(val_imgs)} images")
    return yaml_path

print("Functions defined.")

Functions defined.


In [4]:
# ---------------------------------------------------------------------------
# Paths — Kaggle read-only input on the left, preserved output on the right
# ---------------------------------------------------------------------------

IMAGES_DIR = Path("/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images")
OUT_DIR    = Path("/kaggle/working/weapon_dataset")
RUNS_DIR   = "/kaggle/working/runs/weapon"   # best.pt ends up here
VAL_RATIO  = 0.1

assert IMAGES_DIR.exists(), f"Images directory not found: {IMAGES_DIR}\nCheck that the dataset is attached to this notebook."
print(f"Images dir: {IMAGES_DIR}")
print(f"Output dir: {OUT_DIR}")
print(f"XML count:  {len(list(IMAGES_DIR.glob('*.xml')))}")
print(f"JPG count:  {len(list(IMAGES_DIR.glob('*.jpg')))}")

Images dir: /kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images
Output dir: /kaggle/working/weapon_dataset
XML count:  5149
JPG count:  5149


In [5]:
# ---------------------------------------------------------------------------
# Step 1 — Collect paired image/annotation files
# ---------------------------------------------------------------------------

print("=== STEP 1: Collect image/annotation pairs ===")
pairs = collect_pairs(IMAGES_DIR)
print(f"  Total: {len(pairs)} pairs")

=== STEP 1: Collect image/annotation pairs ===
  Images: 5149 image/annotation pairs found
  Total: 5149 pairs


In [6]:
# ---------------------------------------------------------------------------
# Step 2 — Convert Pascal VOC XML → YOLO txt, copy images to working dir
# ---------------------------------------------------------------------------

print("=== STEP 2: Convert Pascal VOC → YOLO ===")
out_img_dir = OUT_DIR / "images" / "all"
out_lbl_dir = OUT_DIR / "labels" / "all"
counts = convert_and_copy(pairs, out_img_dir, out_lbl_dir)
print(f"  Positive frames (with objects): {counts['pos']}")
print(f"  Negative frames (no objects):   {counts['neg']}")
for cls in CLASSES:
    print(f"    {cls}: {counts[cls]} instances")

=== STEP 2: Convert Pascal VOC → YOLO ===


Converting annotations:   0%|          | 0/5149 [00:00<?, ?img/s]

  Positive frames (with objects): 1534
  Negative frames (no objects):   3615
    gun: 1714 instances
    rifle: 797 instances
    knife: 210 instances


In [7]:
# ---------------------------------------------------------------------------
# Step 3 — Build 90/10 train/val split and write YOLO dataset YAML
# ---------------------------------------------------------------------------

print("=== STEP 3: Build train/val split ===")
yaml_path = write_yaml(OUT_DIR, VAL_RATIO)

=== STEP 3: Build train/val split ===


Copying train:   0%|          | 0/4635 [00:00<?, ?img/s]

Copying val:   0%|          | 0/514 [00:00<?, ?img/s]

  YAML: /kaggle/working/weapon_dataset/weapon.yaml
  Train: 4635 images
  Val:   514 images


In [8]:
# ---------------------------------------------------------------------------
# Step 4 — Train YOLO11n (150 epochs, 640px, T4 x2 GPUs)
# best.pt saved to /kaggle/working/runs/weapon/train/weights/best.pt
# ---------------------------------------------------------------------------

from ultralytics import YOLO

print("=== STEP 4: Train YOLO11n ===")
model = YOLO("yolo11n.pt")
model.train(
    data=str(yaml_path),
    epochs=150,
    imgsz=640,
    batch=128,            # doubled for 2× T4 (each handles 64)
    device=[0, 1],        # use both Kaggle T4 GPUs
    project=RUNS_DIR,
    name="train",
    exist_ok=True,
    # Augmentation tuned for CCTV: slight rotation, no vertical flip, colour jitter
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.5,
    degrees=5,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
)

best = Path(RUNS_DIR) / "train" / "weights" / "best.pt"
print(f"\n{'='*60}")
print(f"Training complete!")
print(f"Best weights: {best}")
print(f"\nDownload best.pt from the Output tab, then:")
print(f"  cp best.pt backend/models/weapon.pt")
print(f"{'='*60}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
=== STEP 4: Train YOLO11n ===
Ultralytics 8.4.19 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
                                                      CUDA:1 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/weapon_dataset/weapon.yaml, degrees=5, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.